In [1]:
from sqlalchemy import create_engine
from collections import Counter
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

import os
import random
import time
import pandas as pd
import re


In [2]:
def SeleniumGetPhoto(URL):
    
    CHROMEDRIVER_PATH = './tools/chromedriver.exe'
    driver = webdriver.Chrome(CHROMEDRIVER_PATH)
    driver.get("https://www.17ce.com/")
    time.sleep(3)
    driver.maximize_window()
    time.sleep(2)
    driver.find_element(By.ID, "nav1").click()
    driver.find_element(By.ID, "url").click()
    driver.find_element(By.ID, "url").send_keys(URL)
    driver.find_element(By.ID, "su").click()
    web = URL.replace("https://","").replace("/","")
    time.sleep(60)
    soup = BeautifulSoup(driver.page_source,"html.parser")
#     driver.close()

    return web , soup

#----------------------------------------------------------------------------

def RequestsGet(web,soup):
    
    line = []
    fastestnode = []
    fastestsec = []
    slowestnode = []
    slowestsec = []
    average = []
    
    table = soup.find("tbody")
    
    for i in table.find_all("tr")[:6]:
        line.append(i.find_all("td")[0].text)
        fastestnode.append(i.find_all("td")[1].find("font",class_="fl").text)
        fastestsec.append(i.find_all("td")[1].find("font",class_="fr").text)
        slowestnode.append(i.find_all("td")[2].find("font",class_="fl").text)
        slowestsec.append(i.find_all("td")[2].find("font",class_="fr").text)
        average.append(i.find_all("td")[3].find("font",class_="fr").text)

    result_dict = {
        "URL":web,
        "Date":time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),
        "Line":line,
        "Fastest_node":fastestnode,
        "Fastest_node_seconds":fastestsec,
        "Slowest_node":slowestnode,
        "Slowest_node_seconds":slowestsec,
        "Average_response":average

    }

    result = pd.DataFrame(result_dict,columns=["URL","Date","Line","Fastest_node","Fastest_node_seconds","Slowest_node","Slowest_node_seconds","Average_response"])
    
    return result



#----------------------------------------------------------------------------


user = "root"
passwd = "IqWYF(TD2020"
ip = "192.168.56.112:3306"
db_name = "Stock"
table_name = "Stock_information"

#連結資料庫
#範本解釋:create_engine('mysql+mysql_driver://mysql帳號:mysql密碼@機器ip:mysql_port/DB名稱?其他參數', encoding='mysql編碼'
#charset=utf8 資料庫編碼

def List_to_mysql(user,passwd,ip,db_name,table_name):
#     user = "root"
#     passwd = "IqWYF(TD2020"
#     ip = "192.168.56.112:3306"
#     db_name = "Stock"
#     table_name = "Stock_information"

    engine = create_engine('mysql+mysqlconnector://'+ user +':'+ passwd +'@'+ip+'/'+ db_name +'?charset=utf8', encoding='utf-8')
    con = engine.connect() #建立連結

    for item in stock_list:
        df = pd.DataFrame(item, index=[0])
        try:
            df.to_sql(name=table_name,con=con,if_exists='append',index=False) #假設table已存在 就自動往下加入data

        except Exception as e:
            if 'PRIMARY' in str(e):
                pass

    con.close() 
    engine.dispose()


    

In [3]:
URL = "https://smzb.io/"
web , soup = SeleniumGetPhoto(URL)
result = RequestsGet(web,soup)


In [4]:
node_list = []
node_table = soup.find("table",{"id":"tblSort"})
table_tr = node_table.find_all("tr")

count = 0

for tr in table_tr[2:]:
    count += 1
    
    for number,content in enumerate(tr):

        if content.text == "":
            continue

        else:

            if number == 0:
                Node = content.text

            elif number == 1:
                ISP = content.text

            elif number == 2:
                Province = content.text

            elif number == 3:
                IP = content.text

            elif number == 4:
                DNS_position = content.text

            elif number == 5:
                Ststus = content.text

            elif number == 6:
                Total_time = content.text

            elif number == 7:
                Resolution_time = content.text

            elif number == 8:
                Connection_time = content.text

            elif number == 9:
                Download_time = content.text

            elif number == 10:
                First_byte_time = content.text

            elif number == 11:
                File_size = content.text

            elif number == 12:
                Download_size = content.text

            elif number == 13:
                Download_speed = content.text

    node_dict = {
        "URL" : URL,
        "Date" : time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),
        "Node" : Node,
        "ISP" : ISP,
        "Province" : Province,
        "IP" : IP,
        "DNS_position" : DNS_position,
        "Ststus" : Ststus,
        "Total_time" : Total_time,
        "Resolution_time" : Resolution_time,
        "Connection_time" : Connection_time,
        "Download_time" : Download_time,
        "First_byte_time" : First_byte_time,
        "File_size" : File_size,
        "Download_size" : Download_size,
        "Download_speed" : Download_speed

    }
    
    node_list.append(node_dict)

    
        
        


In [5]:
len(node_list)

452

In [7]:
df = pd.DataFrame(node_list)

In [11]:
df.iloc[2:4]

,Connection_time,DNS_position,Date,Download_size,Download_speed,Download_time,File_size,First_byte_time,IP,ISP,Node,Province,Resolution_time,Ststus,Total_time,URL
2,0.205s,CLOUDFLAREcloudflare.com,2020-05-27 11:45:53,1.802KB,1.551KB/s,1.161419s,1.802KB,0.958s,104.22.35.126,移动,天津天津市移动,天津,0.000s,200,2.120s,https://smzb.io/
3,0.205s,CLOUDFLAREcloudflare.com,2020-05-27 11:45:53,1.802KB,1.551KB/s,1.161419s,1.802KB,0.958s,104.22.35.126,移动,天津天津市移动,天津,0.000s,200,2.120s,https://smzb.io/
